# Importing scores

In [1]:
import pandas as pd

data_sc = pd.read_csv('final scores 3000.csv')
data_alp = pd.read_excel('data_scoring_initial.xlsx')

In [2]:
data_alp.head()

,Product,Volumes,Demand,Costs,Alphas,Price
0,1,5.63,2,26,"[0.0, 0.26, 0.21, 0.24, 0.16, 0.41, 0.27, 0.8,...",39.0
1,2,2.79,9,29,"[0.67, 0.0, 0.63, 0.18, 0.35, 0.06, 0.23, 0.32...",43.5
2,3,2.93,11,97,"[0.58, 0.85, 0.0, 0.64, 0.44, 0.09, 0.48, 0.1,...",145.5
3,4,3.50,10,90,"[0.36, 0.57, 0.49, 0.0, 0.27, 0.56, 0.2, 0.21,...",135.0
4,5,4.23,14,27,"[0.38, 0.45, 0.35, 0.05, 0.0, 0.18, 0.48, 0.31...",40.5


In [3]:
import ast
data_alp['Alphas'] = data_alp['Alphas'].apply(ast.literal_eval)
print(type(data_alp['Alphas'][0]))

# Check the length of the first list in the 'Alphas' column
print(len(data_alp['Alphas'][0]))

<class 'list'>
3000


In [4]:
data_sc['Alphas'] = data_alp['Alphas']

In [6]:
data = data_sc[['Product', 'Volumes', 'Demand', 'Alphas', 'Costs', 'Price', 'Scores']]

In [7]:
data.head()

,Product,Volumes,Demand,Alphas,Costs,Price,Scores
0,1,5.63,2,"[0.0, 0.26, 0.21, 0.24, 0.16, 0.41, 0.27, 0.8,...",26,39.0,0.003061
1,2,2.79,9,"[0.67, 0.0, 0.63, 0.18, 0.35, 0.06, 0.23, 0.32...",29,43.5,0.003061
2,3,2.93,11,"[0.58, 0.85, 0.0, 0.64, 0.44, 0.09, 0.48, 0.1,...",97,145.5,0.255343
3,4,3.50,10,"[0.36, 0.57, 0.49, 0.0, 0.27, 0.56, 0.2, 0.21,...",90,135.0,0.107907
4,5,4.23,14,"[0.38, 0.45, 0.35, 0.05, 0.0, 0.18, 0.48, 0.31...",27,40.5,0.003061


Products ranking

In [8]:
data = data.sort_values(by='Scores', ascending=False)

In [9]:
df = data.head(100)

In [10]:
alphas = list(df['Alphas'])

In [11]:
df.head()

,Product,Volumes,Demand,Alphas,Costs,Price,Scores
1520,1521,3.93,14,"[0.87, 0.02, 0.33, 0.82, 0.66, 0.2, 0.31, 0.59...",96,144.0,0.528166
1229,1230,6.28,7,"[0.88, 0.54, 0.12, 0.73, 0.68, 0.33, 0.31, 0.5...",97,145.5,0.510287
2817,2818,1.77,2,"[0.11, 0.21, 0.76, 0.21, 0.24, 0.7, 0.04, 0.39...",100,150.0,0.506592
1129,1130,0.80,15,"[0.17, 0.37, 0.76, 0.87, 0.47, 0.57, 0.01, 0.0...",100,150.0,0.503305
2148,2149,2.64,4,"[0.1, 0.08, 0.77, 0.5, 0.44, 0.2, 0.68, 0.18, ...",94,141.0,0.466261


# The heuristic: include the high ranked

In [12]:
def optimal_assortment_with_u(scores, demands, alpha_matrix, shelf_constraint, K_T, delta, assortment_limit):
    # Step 1: Rank the products in descending order based on their scores
    sorted_indices = list(np.array(df['Product'])-1)
    print(sorted_indices)
    
    # Step 2: Select products i from the ranked list until the shelf constraint C is reached
    assortment = []
    u_values = {}  # Dictionary to store the values of u_i for each product i
    remaining_shelf_constraint = shelf_constraint
    
    for i in sorted_indices:
        u_i = K_T * demands[i]
        
        # Check if selecting product i exceeds the shelf constraint C
        if u_i <= remaining_shelf_constraint:
            assortment.append(i)
            u_values[i] = u_i
            remaining_shelf_constraint -= u_i
            if len(assortment) == assortment_limit:  # Check if assortment limit is reached
                break
    
    # Step 3 and 4: Update u_i to include assortment-based substitution terms
    while True:
        updated_assortment = []
        for i in assortment:
            u_i = K_T * demands[i] + delta * K_T * sum(alpha_matrix[sorted_indices.index(j)][sorted_indices.index(i)] * demands[j] for j in sorted_indices if j not in assortment)
            
            # Check if the updated u_i exceeds the shelf constraint C
            if u_i <= shelf_constraint:
                updated_assortment.append(i)
                u_values[i] = u_i
                shelf_constraint -= u_i
        # Check if an equilibrium is reached (no changes in the assortment)
        if assortment == updated_assortment:
            break
        
        assortment = updated_assortment
    
    res = []
    for i in assortment:
        res.append(((i, u_values[i])))
    
    # Step 5: Return the final assortment set A and the values of u_i
    return res


In [ ]:
import numpy as np
C = 100*np.dot(np.array(df['Demand']), np.array(df['Volumes']))

In [ ]:
K_T = 100
delta_value = 0.5
limit_assort = 20
final_res = optimal_assortment_with_u(df['Scores'], df['Demand'], alphas, C, K_T, delta_value, limit_assort )